In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"Asia/Dubai\"")

In [ ]:
hex_project_id = _hex_json.loads("\"1f4e03b9-3e36-41f9-bd87-7c279ad7c462\"")

In [ ]:
hex_project_name = _hex_json.loads("\"COURSERA GP: Snowflake for Data Science: Intro to Snowpark ML for Python_learner Guide_Final\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In [ ]:

from snowflake.snowpark.version import VERSION
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import DecimalType


import snowflake.ml.modeling.preprocessing as snowml
from snowflake.ml.modeling.pipeline import Pipeline


import numpy as np
import pandas as pd


import joblib


import warnings; warnings.simplefilter('ignore')

In [ ]:
import hextoolkit
hex_snowflake_conn = hextoolkit.get_data_connection('Diamond_Connection')
session = hex_snowflake_conn.get_snowpark_session()

In [ ]:
# Verify connectivity to Snowflake
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : ADITYAAP56
Role                        : "ACCOUNTADMIN"
Database                    : "DIAMONDS"
Schema                      : "PUBLIC"
Warehouse                   : "PC_HEX_WH"
Snowflake version           : 8.27.0
Snowpark for Python version : 1.19.0


In [ ]:

DIAMONDS_TABLE = 'diamond'
input_tbl = f"{session.get_current_database()}.{session.get_current_schema()}.{DIAMONDS_TABLE}"

In [ ]:

diamonds_df = session.table(input_tbl)


diamonds_df.show()



diamonds_df.describe()

----------------------------------------------------------------------------------------------------
|"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"PRICE"  |"X"   |"Y"   |"Z"   |"CUT"      |"DEPTH"  |
----------------------------------------------------------------------------------------------------
|0.23     |E        |SI2        |55.0         |326      |3.95  |3.98  |2.43  |IDEAL      |61.5     |
|0.21     |E        |SI1        |61.0         |326      |3.89  |3.84  |2.31  |PREMIUM    |59.8     |
|0.23     |E        |VS1        |65.0         |327      |4.05  |4.07  |2.31  |GOOD       |56.9     |
|0.29     |I        |VS2        |58.0         |334      |4.20  |4.23  |2.63  |PREMIUM    |62.4     |
|0.31     |J        |SI2        |58.0         |335      |4.34  |4.35  |2.75  |GOOD       |63.3     |
|0.24     |J        |VVS2       |57.0         |336      |3.94  |3.96  |2.48  |VERY_GOOD  |62.8     |
|0.24     |I        |VVS1       |57.0         |336      |3.95  |3.98  |2.47  |VERY_GOOD  |6

In [ ]:

diamonds_df.to_pandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53920 entries, 0 to 53919
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CARAT      53920 non-null  float64
 1   COLOR      53920 non-null  object 
 2   CLARITY    53920 non-null  object 
 3   TABLE_PCT  53920 non-null  float64
 4   PRICE      53920 non-null  int16  
 5   X          53920 non-null  float64
 6   Y          53920 non-null  float64
 7   Z          53920 non-null  float64
 8   CUT        53920 non-null  object 
 9   DEPTH      53920 non-null  float64
dtypes: float64(6), int16(1), object(3)
memory usage: 3.8+ MB


In [ ]:
import snowflake.ml.modeling.preprocessing as snowml

categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}
snowml_oe = snowml.OrdinalEncoder(input_cols=["CUT", "CLARITY", "COLOR"], 
                                    output_cols=["CUT_OE", "CLARITY_OE","COLOR_OE"], 
                                    categories=categories
                                )

ord_encoded_diamonds_df = snowml_oe.fit(diamonds_df).transform(diamonds_df)

print(snowml_oe._state_pandas)

   _COLUMN_NAME  _CATEGORY  _INDEX
0           CUT      IDEAL       0
1           CUT    PREMIUM       1
2           CUT  VERY_GOOD       2
3           CUT       GOOD       3
4           CUT       FAIR       4
5       CLARITY         IF       0
6       CLARITY       VVS1       1
7       CLARITY       VVS2       2
8       CLARITY        VS1       3
9       CLARITY        VS2       4
10      CLARITY        SI1       5
11      CLARITY        SI2       6
12      CLARITY         I1       7
13      CLARITY         I2       8
14      CLARITY         I3       9
15        COLOR          D       0
16        COLOR          E       1
17        COLOR          F       2
18        COLOR          G       3
19        COLOR          H       4
20        COLOR          I       5
21        COLOR          J       6


In [ ]:

ord_encoded_diamonds_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------
|"CUT_OE"  |"CLARITY_OE"  |"COLOR_OE"  |"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"PRICE"  |"X"   |"Y"   |"Z"   |"CUT"      |"DEPTH"  |
-------------------------------------------------------------------------------------------------------------------------------------------
|0.0       |6.0           |1.0         |0.23     |E        |SI2        |55.0         |326      |3.95  |3.98  |2.43  |IDEAL      |61.5     |
|1.0       |5.0           |1.0         |0.21     |E        |SI1        |61.0         |326      |3.89  |3.84  |2.31  |PREMIUM    |59.8     |
|3.0       |3.0           |1.0         |0.23     |E        |VS1        |65.0         |327      |4.05  |4.07  |2.31  |GOOD       |56.9     |
|1.0       |4.0           |5.0         |0.29     |I        |VS2        |58.0         |334      |4.20  |4.23  |2.63  |PREMIUM    |62.4     |
|3.0       |6.0     

SNOWPARK ML Modeling preprocessing [Documentation](https://docs.snowflake.com/developer-guide/snowpark-ml/snowpark-ml-modeling)



In [ ]:
from snowflake.ml.modeling.preprocessing import MinMaxScaler

In [ ]:
snowml_mms = MinMaxScaler(input_cols=["CARAT"],output_cols=["CARAT_NORM"])


normalized_diamonds_df = snowml_mms.fit(diamonds_df).transform(diamonds_df)

normalized_diamonds_df.show()
normalized_diamonds_df.describe()

---------------------------------------------------------------------------------------------------------------------------
|"CARAT_NORM"          |"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"PRICE"  |"X"   |"Y"   |"Z"   |"CUT"      |"DEPTH"  |
---------------------------------------------------------------------------------------------------------------------------
|0.006237006237006237  |0.23     |E        |SI2        |55.0         |326      |3.95  |3.98  |2.43  |IDEAL      |61.5     |
|0.002079002079002079  |0.21     |E        |SI1        |61.0         |326      |3.89  |3.84  |2.31  |PREMIUM    |59.8     |
|0.006237006237006237  |0.23     |E        |VS1        |65.0         |327      |4.05  |4.07  |2.31  |GOOD       |56.9     |
|0.018711018711018705  |0.29     |I        |VS2        |58.0         |334      |4.20  |4.23  |2.63  |PREMIUM    |62.4     |
|0.02286902286902287   |0.31     |J        |SI2        |58.0         |335      |4.34  |4.35  |2.75  |GOOD       |63.3     |
|0.00831

In [ ]:
# Categorize all the features for processing
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] # To name the ordinal encoded columns

categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}

In [ ]:
# Build the pipeline
preprocessing_pipeline = Pipeline(
    steps=[
            (
                "OE",
                snowml.OrdinalEncoder(
                    input_cols=CATEGORICAL_COLUMNS,
                    output_cols=CATEGORICAL_COLUMNS_OE,
                    categories=categories,
                )
            )
    ]
)

PIPELINE_FILE = 'preprocessing_pipeline.joblib'

# Lets create joblib file of the pipeline and save it locally


In [ ]:
joblib.dump(preprocessing_pipeline,PIPELINE_FILE)

['preprocessing_pipeline.joblib']

In [ ]:
# You can also save the pipeline joblibfile file into a Snowflake Internal Stage
session.file.put(PIPELINE_FILE,"MODEL_FILES",overwrite = True)

[PutResult(source='preprocessing_pipeline.joblib', target='preprocessing_pipeline.joblib.gz', source_size=1837, target_size=1024, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]

#### _a- Import Libraries_



In [ ]:
# Snowpark ML
from snowflake.ml.modeling.xgboost import XGBRegressor
# For All Snowpark ML Modeling Classes visit https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling

# joblib
import joblib

In [ ]:
# Load Diamonds Table from Snowflake
#diamonds_df = session.table(input_tbl)
diamonds_df.show()

----------------------------------------------------------------------------------------------------
|"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"PRICE"  |"X"   |"Y"   |"Z"   |"CUT"      |"DEPTH"  |
----------------------------------------------------------------------------------------------------
|0.23     |E        |SI2        |55.0         |326      |3.95  |3.98  |2.43  |IDEAL      |61.5     |
|0.21     |E        |SI1        |61.0         |326      |3.89  |3.84  |2.31  |PREMIUM    |59.8     |
|0.23     |E        |VS1        |65.0         |327      |4.05  |4.07  |2.31  |GOOD       |56.9     |
|0.29     |I        |VS2        |58.0         |334      |4.20  |4.23  |2.63  |PREMIUM    |62.4     |
|0.31     |J        |SI2        |58.0         |335      |4.34  |4.35  |2.75  |GOOD       |63.3     |
|0.24     |J        |VVS2       |57.0         |336      |3.94  |3.96  |2.48  |VERY_GOOD  |62.8     |
|0.24     |I        |VVS1       |57.0         |336      |3.95  |3.98  |2.47  |VERY_GOOD  |6

In [ ]:
# Load the preprocessing pipeline object
#preprocessing_pipeline = joblib.load('preprocessing_pipeline.joblib')

# Visualise the Pipeline 
preprocessing_pipeline.to_sklearn()

Pipeline(steps=[('OE',
 ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 ['CUT', 'COLOR',
 'CLARITY'])]))]) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. Pipeline Pipeline(steps=[('OE',
 ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 ['CUT', 'COLOR',
 'CLARITY'])]))]) OE: ColumnTransformer ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 ['CUT', 'COLOR', 'CLARITY'])]) OE ['CUT', 'COLOR', 'CLARITY'] OrdinalEncoder OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}) remainder passthrough passthrough

In [ ]:
# Categorize all the features for modeling
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] 
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

LABEL_COLUMNS = ['PRICE']
OUTPUT_COLUMNS = ['PREDICTED_PRICE']

In [ ]:
# Split the data into train and test sets
diamonds_train_df, diamonds_test_df = diamonds_df.random_split(weights=[0.9,0.1], seed = 0)

# Run the train and test sets through the Pipeline object we defined earlier
train_df = preprocessing_pipeline.fit(diamonds_train_df).transform(diamonds_train_df)
test_df = preprocessing_pipeline.fit(diamonds_test_df).transform(diamonds_test_df)

In [ ]:
# Define the XGBRegressor
regressor = XGBRegressor(
    input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)

# Train
regressor.fit(train_df)

# Predict
result = regressor.predict(test_df)

In [ ]:
from snowflake.ml.modeling.metrics import r2_score

# Show Result
result.select("PRICE", "PREDICTED_PRICE").show()

# Model Accuracy using R-2 Score
print('Acccuracy:', r2_score(df=result,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))

--------------------------------
|"PRICE"  |"PREDICTED_PRICE"   |
--------------------------------
|351      |347.8470764160156   |
|353      |298.98370361328125  |
|355      |384.3904113769531   |
|357      |397.2026062011719   |
|554      |569.7911376953125   |
|554      |519.9287719726562   |
|2757     |2751.294921875      |
|2759     |2976.74169921875    |
|2759     |2904.55029296875    |
|2762     |2822.248779296875   |
--------------------------------

Acccuracy: 0.981374608979638


In [ ]:
import altair
chart_result = altair.Chart.from_json("""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "transform": [],
                    "layer": [
                        {
                            "description": "scatter point mark layer",
                            "mark": {
                                "type": "point",
                                "filled": true,
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {}
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "PRICE",
                            "type": "quantitative",
                            "title": "PRICE",
                            "scale": {
                                "padding": 8
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "y": {
                            "field": "PREDICTED_PRICE",
                            "type": "quantitative",
                            "title": "PREDICTED_PRICE",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "value": "#4C78A8"
                        },
                        "opacity": {
                            "value": 1
                        }
                    }
                },
                {
                    "description": "scatter tooltip point mark layer",
                    "mark": {
                        "type": "point",
                        "filled": true,
                        "clip": true,
                        "cursor": "crosshair"
                    },
                    "encoding": {
                        "x": {
                            "field": "PRICE",
                            "type": "quantitative",
                            "title": "PRICE",
                            "scale": {
                                "padding": 8
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "y": {
                            "field": "PREDICTED_PRICE",
                            "type": "quantitative",
                            "title": "PREDICTED_PRICE",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "value": "#4C78A8"
                        },
                        "tooltip": [
                            {
                                "field": "PRICE",
                                "type": "quantitative",
                                "formatType": "NUMBER_FORMATTER",
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "title": "PRICE"
                            },
                            {
                                "field": "PREDICTED_PRICE",
                                "type": "quantitative",
                                "formatType": "NUMBER_FORMATTER",
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "title": "PREDICTED_PRICE"
                            }
                        ],
                        "opacity": {
                            "value": 0,
                            "condition": {
                                "param": "point_hover_a09c5c92-18b3-4242-82e0-4957a55f47cd",
                                "value": 1,
                                "empty": false
                            }
                        },
                        "size": {
                            "value": 100
                        }
                    },
                    "params": [
                        {
                            "name": "point_hover_a09c5c92-18b3-4242-82e0-4957a55f47cd",
                            "select": {
                                "type": "point",
                                "on": "mouseover",
                                "nearest": false
                            }
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "filter": "isValid(datum[\"PRICE\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_result.datasets.layer00 = result.to_json(orient='records')
chart_result.display(actions=False)

In [ ]:
# Import LinearRegression 
from snowflake.ml.modeling.linear_model import LinearRegression

In [ ]:
# Define the LinearRegression
linear_regressor = LinearRegression(
    input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
    
)


# Train
linear_regressor.fit(train_df)


# Predict
linear_regressor.predict(test_df)

In [ ]:
# Show Result   

result.select("PRICE", "PREDICTED_PRICE").show()

print('Acccuracy:', r2_score(df=result,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))


--------------------------------
|"PRICE"  |"PREDICTED_PRICE"   |
--------------------------------
|351      |347.8470764160156   |
|353      |298.98370361328125  |
|355      |384.3904113769531   |
|357      |397.2026062011719   |
|554      |569.7911376953125   |
|554      |519.9287719726562   |
|2757     |2751.294921875      |
|2759     |2976.74169921875    |
|2759     |2904.55029296875    |
|2762     |2822.248779296875   |
--------------------------------

Acccuracy: 0.981374608979638


In [ ]:
#Import Libraries
from snowflake.ml.modeling.model_selection import GridSearchCV


In [ ]:


grid_search = GridSearchCV(
    estimator=regressor, # same XGBOOST model which we trained earlier
    param_grid={
        "n_estimators":[100,200,300, 400, 500], # Number of boosting rounds
        "learning_rate":[0.1,0.2,0.3, 0.4, 0.5], # Step size shrinkage to prevent overfitting
    },
    n_jobs = -1, #Number of jobs to run in parallel. -1 means using all processors.
    scoring="r2",
    cv=5, # 5-fold cross-validation
    input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)

# Train hypertuned model 
grid_search.fit(train_df)


In [ ]:
# Best Estimator (model) Hyperparameters
grid_search.to_sklearn().best_params_

{'learning_rate': 0.2, 'n_estimators': 500}

In [ ]:
# Predict Price using Test set using best estimator (model)
result_optimal_model = grid_search.predict(test_df)

# Analyze results using R-2 Score 
print('Acccuracy of Hypertuned XGBOOST Model:', r2_score(df=result_optimal_model,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))
print('Acccuracy of Initial XGBOOST Model:', r2_score(df=result,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))

Acccuracy of Hypertuned XGBOOST Model: 0.9814792891629546
Acccuracy of Initial XGBOOST Model: 0.981374608979638


In [ ]:
#import libraries 
import joblib
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import OrdinalEncoder
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.snowpark import types as T
import snowflake.snowpark.functions as F
import cachetools
import pandas as pd

In [ ]:
# Categorize all the features for modeling
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"]
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}

# Define the end to end pipeline that contains Transforms (preprocessing) and Estimator (optimal XGBRegressor model)
pipe = Pipeline(
    steps=[
        ("OE", OrdinalEncoder(
            input_cols= CATEGORICAL_COLUMNS,  
            output_cols= CATEGORICAL_COLUMNS_OE, 
            categories = categories,  
            drop_input_cols=True)
            ),
        ("regressor", XGBRegressor(
            learning_rate = 0.2, # Add Best best_params_ Results here 
            n_estimators = 500,  # Add Best best_params_ Results here 
            input_cols=CATEGORICAL_COLUMNS_OE + NUMERICAL_COLUMNS, 
            label_cols=["PRICE"], 
            output_cols=['PREDICTED_PRICE'], 
            n_jobs=-1)
            )
    ]
)

# Random split
diamonds_train_df, diamonds_test_df = diamonds_df.random_split(weights=[0.9,0.1], seed =0)

# Train the model and convert it to sklearn pipeline
xgb_optimal_model = pipe.fit(diamonds_train_df).to_sklearn()

In [ ]:
from sklearn import set_config
set_config(display="diagram")
xgb_optimal_model

Pipeline(steps=[('OE',
 ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 [7, 1, 2])])),
 ('filter...
 feature_types=None, gamma=None, gpu_id=None,
 grow_policy=None, importance_type=None,
 interaction_constraints=None, learning_rate=0.2,
 max_bin=None, max_cat_threshold=None,
 max_cat_to_onehot=None, max_delta_step=None,
 max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan,
 monotone_constraints=None, n_estimators=500,
 n_jobs=-1, num_parallel_tree=None, predictor=None,
 random_state=None, ...))]) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. Pipeline Pipeline(steps=[('OE',
 ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 [7, 1, 2])])),
 ('filter...
 feature_types=None, gamma=None, gpu_id=None,
 grow_policy=None, importance_type=None,
 interaction_constraints=None, learning_rate=0.2,
 max_bin=None, max_cat_threshold=None,
 max_cat_to_onehot=None, max_delta_step=None,
 max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan,
 monotone_constraints=None, n_estimators=500,
 n_jobs=-1, num_parallel_tree=None, predictor=None,
 random_state=None, ...))]) OE: ColumnTransformer ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 [7, 1, 2])]) OE [7, 1, 2] OrdinalEncoder OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}) remainder ['CARAT', 'TABLE_PCT', 'X', 'Y', 'Z', 'DEPTH'] passthrough passthrough filter_input_cols_for_estimator: ColumnTransformer ColumnTransformer(remainder='passthrough',
 transformers=[('filter_input_cols_for_estimator',
 'passthrough', [0, 1, 2, 3, 8, 4, 5, 6, 7])]) filter_input_cols_for_estimator [0, 1, 2, 3, 8, 4, 5, 6, 7] passthrough passthrough remainder [] passthrough passthrough XGBRegressor XGBRegressor(base_score=None, booster=None, callbacks=None,
 colsample_bylevel=None, colsample_bynode=None,
 colsample_bytree=None, early_stopping_rounds=None,
 enable_categorical=False, eval_metric=None, feature_types=None,
 gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
 interaction_constraints=None, learning_rate=0.2, max_bin=None,
 max_cat_threshold=None, max_cat_to_onehot=None,
 max_delta_step=None, max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan, monotone_constraints=None,
 n_estimators=500, n_jobs=-1, num_parallel_tree=None,
 predictor=None, random_state=None, ...)

In [ ]:

MODEL_FILE = 'xgb_optimal_pipe.joblib'

joblib.dump(xgb_optimal_model, MODEL_FILE)

session.file.put(MODEL_FILE, "@MODEL_FILES", overwrite=True, auto_compress=False)

[PutResult(source='xgb_optimal_pipe.joblib', target='xgb_optimal_pipe.joblib', source_size=2093346, target_size=2093360, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [ ]:

@cachetools.cached(cache={})
def read_file(filename):
    import joblib
    import sys
    import os

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m


@F.udf(name="predict_diamond_price",
        is_permanent=True,
        stage_location = '@MODEL_FILES',
        imports=['@MODEL_FILES/xgb_optimal_pipe.joblib'],
        packages=['snowflake-ml-python', 'joblib', 'scikit-learn==1.2.2', 'xgboost==1.7.3', 'cachetools'],
        replace=True,
        session=session)
def predict_diamond_price(pd_input: T.PandasDataFrame[str, str, str, float, float, float, float, float, float]) -> T.PandasSeries[float]:
    features = ["CUT", "COLOR", "CLARITY", "CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]
    pd_input.columns =  features
    model = read_file("xgb_optimal_pipe.joblib")            
    prediction = model.predict(pd_input)       
    return              

In [ ]:

new_diamonds =  session.table("DIAMONDS_NEW")

new_diamonds_w_prediction = new_diamonds.with_column("PREDICTED_PRICE", F.call_function("predict_diamond_price", 
                                    F.col("CUT"), F.col("COLOR"), F.col("CLARITY"), 
                                    F.col("CARAT"), F.col("DEPTH"), F.col("TABLE_PCT"), 
                                    F.col("X"), F.col("Y"), F.col("Z"))
                                    )

new_diamonds_w_prediction.show()